# LSTM Language Model

In this notebook, we are going to make a Language Model using LSTMs. This is the "old-school" way to make language models. Recently, with the introduction of the Transformer architecture, one can successfully make a Language Model with better overall quality instead of using LSTM.

In [1]:
%load_ext autoreload
%autoreload 2
from practicalnlp import settings
from practicalnlp.models import *
from practicalnlp.training import *
from practicalnlp.data import *
import torch

# Loading data

Here we load all data with `batch_size = 20`. It's important to note that we subdivide data with 2 parameters: `nctx` and `batch_size`. `nctx` is the number of words we are using in a single pass of a training phase. For example, the figure below ilustrates each *step* in the training phase for `nctx = 3` over a single `batch_size` of the entire sentence below.


<img src="training_step_lm.svg" width="800" />
<!--- [svg](training_step_lm.svg)> --->

Arrows indicate that the origin word is trying to predict the next word in the `nctx` window. When the last word of the `nctx` window is processed, the window is translated by `nctx` words and the process repeats until it reads the entire batch. The `nctx` param is also known as `bptt` (*backpropagation through time*), and is the name used in the official PyTorch tutorial for Language Modeling.

Although this example shows the execution for only a single batch, in practice, we do it for all batchs at the same time. It might be easy to understand how it can be done in practice with a 2-dimensional tensor (one dimension for batch size, and other for the sequence length). In the code below, we do it using PyTorch.

In [2]:
batch_size = 20
nctx = 35
TRAIN = settings.WIKI_TRAIN_DATA
VALID = settings.WIKI_VALID_DATA
reader = WordDatasetReader(nctx)
reader.build_vocab((TRAIN,))

train_set = reader.load(TRAIN, batch_size)
valid_set = reader.load(VALID, batch_size)

In [4]:
train_set.shape

torch.Size([20, 104431])

In [3]:
model = LSTMLanguageModel(len(reader.vocab), 512, 512)
model.to('cuda:0')

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {num_params} parameters") 


learnable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(learnable_params, lr=0.001)
fit_lm(model, optimizer, 1, batch_size, nctx, train_set, valid_set)

Model has 21274623 parameters


/workspace/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


EPOCH 1
Training Results
average_train_loss 6.993510 (7.717011)
average_train_loss 6.968325 (7.219686)
average_train_loss 6.436844 (6.992108)
average_train_loss 6.624825 (6.847209)
average_train_loss 6.506425 (6.741867)
average_train_loss 6.232456 (6.661590)
average_train_loss 6.241143 (6.604560)
average_train_loss 6.209402 (6.553402)
average_train_loss 6.053204 (6.508152)
average_train_loss 5.882931 (6.471613)
average_train_loss 6.114008 (6.439420)
average_train_loss 5.837910 (6.412545)
average_train_loss 6.178204 (6.389939)
average_train_loss 6.141484 (6.364485)
average_train_loss 6.193256 (6.346820)
average_train_loss 6.029351 (6.331054)
average_train_loss 6.109805 (6.309967)
average_train_loss 5.808486 (6.289859)
average_train_loss 5.751597 (6.273885)
average_train_loss 5.986725 (6.256851)
average_train_loss 5.706111 (6.239590)
average_train_loss 5.704154 (6.220216)
average_train_loss 5.918287 (6.202540)
average_train_loss 5.850075 (6.190269)
average_train_loss 5.963675 (6.176846)


In [6]:
def sample(model, index2word, start_word='the', maxlen=20):
  

    model.eval() 
    words = [start_word]
    x = torch.tensor(reader.vocab.get(start_word)).long().reshape(1, 1).to('cuda:0')
    hidden = model.init_hidden(1)

    with torch.no_grad():
        for i in range(20):
            output, hidden = model(x, hidden)
            word_softmax = output.squeeze().exp().cpu()
            selected = torch.multinomial(word_softmax, 1)[0]
            x.fill_(selected)
            word = index2word[selected.item()]
            words.append(word)
    words.append('...')
    return words

index2word = {i: w for w, i in reader.vocab.items()}
words = sample(model, index2word)
print(' '.join(words))

the American album were 28 – 3 . During 1907 , suspicions remained the first time since 1972 with him just ...
